In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp anndata_iso

In [ ]:
#| export
import numpy as np
import pandas as pd
import anndata as ad

In [ ]:
from allos.readers_tests import *
from allos.preprocessing import subset_common_cells, transfer_obs

In [ ]:
from allos.readers_tests import *

df = pd.read_csv('e18.mouse.clusters.csv')
df['barcode'] = df.index.str.split('_').str[1]


mouse_data_str_1 = download_test_data(output_filename='mouse_1.txt')
print("Test data downloaded successfully")


mouse_data_str_2 = download_test_data("https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748089/suppl/GSM3748089%5F951c.isoforms.matrix.txt.gz", output_filename='mouse_2.txt')
print("Test data downloaded successfully")


mouse_1 = read_sicelore_isomatrix(file_path=mouse_data_str_1)
mouse_2 = read_sicelore_isomatrix(file_path=mouse_data_str_2)

combined_mouse_data = iso_concat([mouse_1, mouse_2], batch_type='numeric')


combined_mouse_data.obs_names_make_unique()
# Step 1: Remove any duplicate barcodes in the DataFrame
df_unique = df.drop_duplicates(subset='barcode')

# Step 2: Filter the DataFrame to include only the barcodes present in the AnnData object
df_filtered = df_unique[df_unique['barcode'].isin(combined_mouse_data.obs_names)]

# Step 3: Set the index of the filtered DataFrame to 'barcode' to make the merge easier
df_filtered.set_index('barcode', inplace=True)

# Step 4: Create a DataFrame from the obs DataFrame of the AnnData object to ensure the same index
obs_df = combined_mouse_data.obs.copy()

# Step 5: Initialize a new column 'cell_type' with NaN values in the obs DataFrame
obs_df['cell_type'] = pd.NA

# Step 6: Update the 'cell_type' column with values from the filtered DataFrame where indices match
obs_df.update(df_filtered['illumina.ident'].rename('cell_type'))

# Step 7: Ensure the index is unique and assign the updated DataFrame back to the obs attribute of the AnnData object
if obs_df.index.is_unique:
    combined_mouse_data.obs = obs_df
else:
    raise ValueError("The index of the obs DataFrame is not unique.")

# Now, the 'cell_type' column should be added to the obs DataFrame of your AnnData object
combined_mouse_data = combined_mouse_data[~combined_mouse_data.obs['cell_type'].isna()]

Starting download of test data from https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748087/suppl/GSM3748087%5F190c.isoforms.matrix.txt.gz
File downloaded successfully
File extracted successfully
Test data downloaded successfully
Starting download of test data from https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748089/suppl/GSM3748089%5F951c.isoforms.matrix.txt.gz
File downloaded successfully
File extracted successfully
Test data downloaded successfully


/data/analysis/data_mcandrew/Allos_new/allos_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
combined_mouse_data.obs.index

Index(['CCACCTACAAAGTCAA', 'AACCATGAGACCACGA', 'GATGAGGTCGACCAGC',
       'GCACTCTTCTATCGCC', 'TGCCCTAAGGCCCGTT', 'TTCGGTCAGCTGGAAC',
       'GGATTACAGACGCAAC', 'GTCTCGTTCCTCTAGC', 'CCATGTCAGTAGATGT',
       'GATCGATGTCAAACTC',
       ...
       'GCTCCTACATCGACGC', 'AGCGTCGCATTATCTC', 'ATCCACCTCCCACTTG',
       'GTGCTTCCAAGGCTCC', 'GACTACATCATTGCGA', 'TACTCATTCGCCATAA',
       'CGTGAGCTCTCTAAGG', 'TTATGCTAGTAAGTAC', 'CGGACACTCTTGTCAT',
       'GGCTCGAGTACCGAGA'],
      dtype='object', length=1109)

In [ ]:
# | export
class AnnDataIso(ad.AnnData):
    # Private method to filter dataset for genes with multiple isoforms.
    def _filter_isodata(self):
        """
        Filters the data to include only genes with multiple isoforms.

        Logic:
        - Counts the number of isoforms for each gene using `np.unique`.
        - Creates a DataFrame with gene IDs and their isoform counts.
        - Identifies genes with more than one isoform.
        - Subsets the AnnData object to include only these genes.

        Returns:
        - Filtered AnnData object with only genes having multiple isoforms.
        """
        genes, frequencies = np.unique(self.var['geneId'], return_counts=True)
        df = pd.DataFrame({"geneId": genes, "frequencies": frequencies})
        multi_iso_genes = df[df["frequencies"] > 1]['geneId'].tolist()
        filtr_adata_iso = self[:, self.var['geneId'].isin(multi_iso_genes)]
        return filtr_adata_iso

    # Method to compute isoform expression percentages relative to total gene expression.
    def iso_percent(self, df, barcodes_regex="^[ACGT]+$"):
        """
        Calculates isoform expression percentages for each barcode.

        Parameters:
        - df (DataFrame): Input DataFrame containing isoform expression data.
        - barcodes_regex (str): Regex to identify columns representing barcodes.

        Returns:
        - DataFrame with normalized isoform percentages for each barcode.
        """
        iso_perc_df = df.__deepcopy__()
        if len(iso_perc_df.filter(regex=(barcodes_regex)).columns.to_list()) < 1:
            raise ValueError("No barcode was identified. Please check the names of the columns.")
        # Normalize isoform counts by gene-level expression.
        iso_perc_df[iso_perc_df.filter(regex=(barcodes_regex)).columns.to_list()] = (
            iso_perc_df.filter(regex=(barcodes_regex)) /
            iso_perc_df.groupby(['geneId']).transform('sum').filter(regex=(barcodes_regex))
        )
        # Replace NaN values with 0.
        iso_perc_df = iso_perc_df.replace(np.nan, 0.0)
        return iso_perc_df

    # Class initializer to process the AnnData object and compute relevant annotations.
    def __init__(self, anndata: ad.AnnData, palette='ghibli'):
        """
        Initializes the AnnDataIso object with isoform and gene-level annotations.

        Parameters:
        - anndata (AnnData): Input AnnData object containing isoform-level data.
        - cell_types (DataFrame): DataFrame with cell type annotations.
        """
        self._init_as_actual(anndata.copy())
        # Group genes and count the number of isoforms.
        self.gene_counts = self.var.reset_index().groupby(by='geneId').count()
        # Filter dataset for genes with multiple isoforms.
        self._filtered_anndata = self._filter_isodata()
        # Annotate cell types in the observation data.
        # Compute isoform percentages.
        df = self._filtered_anndata.to_df().set_index(self._filtered_anndata.obs['barcodes'])
        df = df.transpose()
        df[['transcriptId', 'geneId']] = self._filtered_anndata.var
        df_m_iso = self.iso_percent(df)
        df_m_iso = df_m_iso.iloc[0:, :-2].transpose()
        self._filtered_anndata.obsm['Iso_prct'] = df_m_iso
        self.colors = ['#6c6246', '#387041', '#153c27', '#393525', '#b19c77', '#175934', '#d0e7f3', '#466175', '#8c8c5a', '#47462c', '#8c414e', '#579f4e', '#7cb282', '#30363a', '#672c37', '#e5c6ad', '#402b2f', '#b0606e', '#c1e5d8', '#829ea7']
        if palette != 'ghibli':
            self.colors = ['#100856', '#7243ee', '#da2ad3', '#3a58a4', '#2d149e', '#7f0d9a', '#3b0554', '#7a97f4', '#a127b4', '#696e43', '#de62e2', '#4e1af5', '#88904e', '#2d2c18', '#576d5c', '#4a4f20', '#909cfa', '#929574', '#c905ca', '#71a493']


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()